Downloading the Iowa Elections Files

1: Setup

In [1]:
from pathlib import Path


ROOT = Path.home() / "Documents" / "STAT3255" / "team_b"

RAW_DIR = ROOT / "data" / "raw"
OUT_DIR = ROOT / "output"

YEAR_DIR = {
    2016: RAW_DIR / "2016",
    2018: RAW_DIR / "2018",
    2020: RAW_DIR / "2020",
}

# create folders
for p in list(YEAR_DIR.values()) + [OUT_DIR]:
    p.mkdir(parents=True, exist_ok=True)

print("ROOT:", ROOT.resolve())
for y, p in YEAR_DIR.items():
    print(f"{y} download folder:", p.resolve())
print("output folder:", OUT_DIR.resolve())


ROOT: /Users/sophiafazzina/Documents/STAT3255/team_b
2016 download folder: /Users/sophiafazzina/Documents/STAT3255/team_b/data/raw/2016
2018 download folder: /Users/sophiafazzina/Documents/STAT3255/team_b/data/raw/2018
2020 download folder: /Users/sophiafazzina/Documents/STAT3255/team_b/data/raw/2020
output folder: /Users/sophiafazzina/Documents/STAT3255/team_b/output


2: Driver Setup

In [2]:
import time
from pathlib import Path

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

def make_driver(download_dir: Path, headless: bool = False):
    download_dir = download_dir.resolve()

    options = webdriver.ChromeOptions()

    
    prefs = {
        "download.default_directory": str(download_dir),
        "download.prompt_for_download": False,
        "download.directory_upgrade": True,
        "safebrowsing.enabled": True,
    }
    options.add_experimental_option("prefs", prefs)

    
    if headless:
        options.add_argument("--headless=new")

    driver = webdriver.Chrome(
        service=Service(ChromeDriverManager().install()),
        options=options
    )
    return driver

def set_download_dir(driver, download_dir: Path):
    """Change Chrome's download folder after the driver is already running."""
    download_dir = download_dir.resolve()
    driver.execute_cdp_cmd(
        "Page.setDownloadBehavior",
        {"behavior": "allow", "downloadPath": str(download_dir)}
    )
    print("Now downloading into:", download_dir)


driver = make_driver(YEAR_DIR[2016], headless=False)
set_download_dir(driver, YEAR_DIR[2016])


Now downloading into: /Users/sophiafazzina/Documents/STAT3255/team_b/data/raw/2016


3: Collect Excel Links

In [3]:
from selenium.webdriver.common.by import By

YEAR_PAGE = {
    2016: "https://sos.iowa.gov/precinct-results-county-2016-general",
    2018: "https://sos.iowa.gov/precinct-results-county-2018-general",
    2020: "https://sos.iowa.gov/precinct-results-county-2020-general",
}

def get_excel_links(driver, year: int, wait_s: float = 2.0):
    """
    Open the year page and return a de-duplicated list of .xls/.xlsx links.
    """
    driver.get(YEAR_PAGE[year])
    time.sleep(wait_s)  

    anchors = driver.find_elements(By.TAG_NAME, "a")
    links = []
    for a in anchors:
        href = a.get_attribute("href")
        if not href:
            continue
        h = href.lower()
        if h.endswith(".xls") or h.endswith(".xlsx"):
            links.append(href)

    
    links = list(dict.fromkeys(links))
    return links

# quick test
for y in [2016, 2018, 2020]:
    links = get_excel_links(driver, y)
    print(y, "links:", len(links), "example:", links[0] if links else "NONE")


2016 links: 101 example: https://sos.iowa.gov/elections/pdf/precinctresults/2016general/adair.xlsx
2018 links: 101 example: https://sos.iowa.gov/elections/pdf/precinctresults/2018general/adair.xls
2020 links: 101 example: https://sos.iowa.gov/elections/pdf/precinctresults/2020general/adair.xlsx


4: Filter Links

In [8]:
from urllib.parse import urlparse

def filename_from_url(url: str) -> str:
    return urlparse(url).path.split("/")[-1]

def looks_like_county_file(url: str, year: int) -> bool:
    """
    Keep links that look like Iowa county files.
    Typical pattern includes /precinctresults/<year>general/<county>.xls(x)
    """
    u = url.lower()
    if not (u.endswith(".xls") or u.endswith(".xlsx")):
        return False
    if str(year) not in u or "general" not in u:
        return False
    if "precinctresults" not in u:
        return False
    return True

def download_year(driver, year: int, sleep_s: float = 0.6):
    year_dir = YEAR_DIR[year]
    set_download_dir(driver, year_dir)

    links = get_excel_links(driver, year)
    links = [u for u in links if looks_like_county_file(u, year)]
    links = list(dict.fromkeys(links))  

    print(f"{year}: filtered to {len(links)} excel links")

    triggered = 0
    skipped = 0

    for i, url in enumerate(links, start=1):
        fname = filename_from_url(url)
        out_path = year_dir / fname

        if out_path.exists():
            skipped += 1
            continue

        driver.get(url)  # triggers download
        triggered += 1

        if i <= 5 or i % 10 == 0 or i == len(links):
            print(f"{year}: processed {i}/{len(links)} (downloaded={triggered}, skipped={skipped})")

        time.sleep(sleep_s)

    # give downloads a moment to finish writing
    time.sleep(2)

    # count what’s actually there
    files = sorted(list(year_dir.glob("*.xls*")))
    print(f"{year}: folder now has {len(files)} .xls/.xlsx files (example: {files[0].name if files else 'NONE'})")
    return files

# run year-by-year
files_2016 = download_year(driver, 2016)
files_2018 = download_year(driver, 2018)
files_2020 = download_year(driver, 2020)


Now downloading into: /Users/sophiafazzina/Documents/STAT3255/team_b/data/raw/2016
2016: filtered to 99 excel links
2016: folder now has 99 .xls/.xlsx files (example: adair.xlsx)
Now downloading into: /Users/sophiafazzina/Documents/STAT3255/team_b/data/raw/2018
2018: filtered to 99 excel links
2018: folder now has 98 .xls/.xlsx files (example: adair.xls)
Now downloading into: /Users/sophiafazzina/Documents/STAT3255/team_b/data/raw/2020
2020: filtered to 99 excel links
2020: processed 2/99 (downloaded=1, skipped=1)
2020: processed 30/99 (downloaded=5, skipped=25)
2020: folder now has 98 .xls/.xlsx files (example: adair.xlsx)


5: Convert to CSV (one per year)

In [4]:
import pandas as pd

def read_excel_smart(path):
    try:
        return pd.read_excel(path, engine="openpyxl")
    except Exception:
        return pd.read_excel(path, engine="xlrd")

def build_year_csv(year: int, out_csv):
    files = sorted(YEAR_DIR[year].glob("*.xls*"))
    if not files:
        raise RuntimeError(f"No Excel files found for {year} in {YEAR_DIR[year]}")

    frames = []
    bad = []

    for f in files:
        try:
            df = read_excel_smart(f)

            if df is None or df.shape[0] == 0:
                bad.append((f.name, "EMPTY_SHEET"))
                continue

            df.columns = [str(c) for c in df.columns]
            df["source_file"] = f.name
            df["year"] = year
            frames.append(df)

        except Exception as e:
            bad.append((f.name, repr(e)))

    if not frames:
        raise RuntimeError(f"All files failed for {year}. Example errors: {bad[:5]}")

    big = pd.concat(frames, ignore_index=True, sort=True)
    out_csv = Path(out_csv)
    out_csv.parent.mkdir(parents=True, exist_ok=True)
    big.to_csv(out_csv, index=False)

    print(f"{year}: wrote {out_csv} with {len(big):,} rows from {len(frames)} files")
    if bad:
        print(f"{year}: {len(bad)} files skipped/failed (showing up to 15):")
        for name, err in bad[:15]:
            print("  ", name, "->", err)

    return big


In [14]:
!pip -q install xlrd==2.0.1


[notice] A new release of pip is available: 25.2 -> 26.0.1
[notice] To update, run: python -m pip install --upgrade pip


Run the pip install above and then restart the kernel. Then re run the other chunks to get the variables back. 

In [12]:
from pathlib import Path

csv_2016 = OUT_DIR / "iowa_precinct_returns_2016.csv"

df2016 = build_year_csv(2016, csv_2016)

2016: wrote /Users/sophiafazzina/Documents/STAT3255/team_b/output/iowa_precinct_returns_2016.csv with 51,975 rows from 99 files


In [15]:
from pathlib import Path

csv_2020 = OUT_DIR / "iowa_precinct_returns_2020.csv"

df2020 = build_year_csv(2020, csv_2020)

2020: wrote /Users/sophiafazzina/Documents/STAT3255/team_b/output/iowa_precinct_returns_2020.csv with 4,343 rows from 98 files


Code to convert the 2018 files from xls to xlsx. I'm doing this so that the files will work with the same combination script that I used for 2016 and 2020.

In [9]:
from pathlib import Path
import pandas as pd
import xml.etree.ElementTree as ET

Y2018_IN  = YEAR_DIR[2018]
Y2018_OUT = RAW_DIR / "2018_xlsx"
Y2018_OUT.mkdir(parents=True, exist_ok=True)

def sniff_kind(p: Path) -> str:
    head = p.read_bytes()[:4096]
    if head.startswith(b"PK\x03\x04"):
        return "xlsx_zip"
    low = head.lower()
    if b"urn:schemas-microsoft-com:office:spreadsheet" in low or b"<workbook" in low:
        return "spreadsheetml"
    if b"<html" in low or b"<!doctype html" in low:
        return "html"
    if head.startswith(b"\xD0\xCF\x11\xE0\xA1\xB1\x1A\xE1"):
        return "xls_binary"
    return "unknown"

def read_spreadsheetml_to_df(p: Path) -> pd.DataFrame:
   
    raw = p.read_bytes()
    text = raw.decode("utf-8", errors="ignore")

    root = ET.fromstring(text)

    def local(tag: str) -> str:
        # '{namespace}Row' -> 'Row'
        return tag.split("}")[-1] if "}" in tag else tag

    rows_out = []
    for row in root.iter():
        if local(row.tag) != "Row":
            continue

        row_vals = []
        current_col = 1

        for cell in row:
            if local(cell.tag) != "Cell":
                continue

            
            idx = None
            for k, v in cell.attrib.items():
                if k.endswith("Index"):
                    try:
                        idx = int(v)
                    except:
                        idx = None
                    break
            if idx is not None and idx > current_col:
                row_vals.extend([""] * (idx - current_col))
                current_col = idx

            data_val = ""
            for child in cell.iter():
                if local(child.tag) == "Data" and child.text is not None:
                    data_val = child.text
                    break

            row_vals.append(data_val)
            current_col += 1

        if any(str(x).strip() != "" for x in row_vals):
            rows_out.append(row_vals)

    if not rows_out:
        raise ValueError("SpreadsheetML parse found 0 rows")

    
    w = max(len(r) for r in rows_out)
    rows_out = [r + [""] * (w - len(r)) for r in rows_out]

    header = [str(h).strip() if str(h).strip() else f"col_{i}" for i, h in enumerate(rows_out[0])]
    data = rows_out[1:] if len(rows_out) > 1 else []
    return pd.DataFrame(data, columns=header)

def convert_one_2018_file(src: Path, out_dir: Path) -> Path:
    kind = sniff_kind(src)
    out_path = out_dir / (src.stem + ".xlsx")

    if out_path.exists():
        return out_path

    if kind == "xlsx_zip":
        out_path.write_bytes(src.read_bytes())
        return out_path

    if kind == "xls_binary":
        df = pd.read_excel(src, engine="xlrd")

    elif kind == "spreadsheetml":
        try:
            df = read_spreadsheetml_to_df(src)
        except Exception:
            
            tables = pd.read_html(src, flavor="lxml")
            if not tables:
                raise
            df = tables[0]

    elif kind == "html":
        tables = pd.read_html(src, flavor="lxml")
        if not tables:
            raise ValueError("No tables found in HTML file")
        df = tables[0]

    else:
        raise ValueError(f"Unknown file type for {src.name}")

    df.to_excel(out_path, index=False, engine="openpyxl")
    return out_path


src_files = sorted(Y2018_IN.glob("*.xls*"))
print("2018 input files found:", len(src_files))

ok, bad = 0, []
for f in src_files:
    try:
        convert_one_2018_file(f, Y2018_OUT)
        ok += 1
    except Exception as e:
        bad.append((f.name, repr(e)))

print("2018 converted to .xlsx:", ok)
print("2018 failed:", len(bad))
if bad:
    print("Example failures (up to 10):")
    for name, err in bad[:10]:
        print("  ", name, "->", err)

print("Converted folder:", Y2018_OUT.resolve())


2018 input files found: 98
2018 converted to .xlsx: 98
2018 failed: 0
Converted folder: /Users/sophiafazzina/Documents/STAT3255/team_b/data/raw/2018_xlsx


In [10]:
YEAR_DIR[2018] = RAW_DIR / "2018_xlsx"

csv_2018 = OUT_DIR / "iowa_precinct_returns_2018.csv"
df2018 = build_year_csv(2018, csv_2018)


2018: wrote /Users/sophiafazzina/Documents/STAT3255/team_b/output/iowa_precinct_returns_2018.csv with 67,320 rows from 98 files
